In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score
import plotly.express as px
import plotly.graph_objects as go
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf    
tf.compat.v1.disable_v2_behavior() # <-- HERE !

In [ ]:
from tensorflow import keras

In [ ]:
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import load_model

In [ ]:
#this is a public token
mapbox_token = 'pk.eyJ1IjoibGl5YW5neWFuZzUxNSIsImEiOiJjbDBuNmM3MjEwdGZjM2t0NHRqbmJidXFjIn0.8O9DnGkHPecl4jjk1ZqQUQ'

px.set_mapbox_access_token(mapbox_token)

## Exploration

In [ ]:
# laod the merged dataset first
merge_original = pd.read_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\merged_data.csv', index_col = 0)
merge_original

In [ ]:
# view overall statistics
merge_original.describe()

In [ ]:
# see the distribution of data
fig = px.histogram(merge_original, x="NO2")
fig.show()

In [ ]:
# see the distribution of data
fig = px.histogram(merge_original, x="facebook_movement")
fig.show()

In [ ]:
# for highly screwed output, remove outliers, as well as remove noise in facebook mobility input
merge = merge_original.loc[(merge_original['NO2'] > 1e-6) & (merge_original['facebook_movement'] > 50)]

In [ ]:
# add in a new column, wind speed
merge['wind_speed'] = pow((merge['u-wind']**2 + merge['v-wind']**2), 0.5)
# merge['wind_speed'] = merge.apply(lambda x: pow(x['u-wind'] **2 + x['v-wind']**2, 0.5),axis=1)

In [ ]:
# add in a new column, taking log10 of n_crisis
merge['log_facebook_movement'] = np.log10(merge['facebook_movement'])

In [ ]:
# add in a new column, taking log10 of NO2, 1e7 was multiplied to avoid negative value after taking log, and higher NO2 still has the higher transformed NO2
merge['log_NO2'] = np.log10(merge['NO2']*1e7)

In [ ]:
merge.describe()

In [ ]:
# calculate pearson correlation
corr = merge.corr()
corr

In [ ]:
# show correlation in heatmap, hover and see clear values
fig = px.imshow(corr, text_auto=True, width=700, height=700)
fig.show()

In [ ]:
fig = px.histogram(merge, x="NO2")
fig.show()

In [ ]:
fig = px.histogram(merge, x="log_NO2")
fig.show()

In [ ]:
fig = px.histogram(merge, x="log_facebook_movement")
fig.show()

In [ ]:
# save the preprocessed data to csv
# merge.to_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\Preprocessed_data_ML.csv')

In [ ]:
# see how facebook mobility changes with time in indonesia, color shows NO2 levels
fig = px.scatter(merge[merge['country']=='Indonesia'], x='date', y='log_facebook_movement', color='log_NO2')
fig.show()

In [ ]:
# see how no2 changes with time in indonesia, color shows facebook mobility
fig = px.scatter(merge[merge['country']=='Indonesia'], x='date', y='log_NO2', color='log_facebook_movement')
fig.show()

## MLP

In [ ]:
# X include all input values
before_normalized_X = merge[['lon','lat','day', 'rainfall','u-wind', 'v-wind', 'wind_speed', 'surface-p', 'dew-pt', '2m-temp', 'facebook_movement', 'log_facebook_movement', 'apple_driving', 'apple_walking', 'haze']]
before_normalized_X

In [ ]:
# normaize inputs to the same scale
sc = StandardScaler()
sc_X = sc.fit(before_normalized_X)
after_normalized_X = sc_X.transform(before_normalized_X)
after_normalized_X

In [ ]:
after_normalized_X.shape

In [ ]:
y = merge[['log_NO2']]
y

In [ ]:
# test = 30% of total data
X_train, X_test, y_train, y_test = train_test_split(after_normalized_X, y, test_size=0.3, random_state=0)

In [ ]:
# construct MLP architecture
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[15]),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1),
])
model.compile(
    optimizer='adam',
    loss='mae',
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    batch_size = 2048,
    epochs = 300,
#    callbacks=[early_stopping], # put your callbacks in a list
#    verbose=0,  # turn off training log
)

In [ ]:
# print history
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
# if a model was trained and saved, can directly load here
model = load_model('MLP_all_mobility.h5')

In [ ]:
# print root mean square error and mean absolute error for testing dataset (original data)
mse_test = mse(pow(10,y_test)*1e-7,pow(10,model.predict(X_test))*1e-7)
print(mse_test**0.5)
print(mae(pow(10,y_test)*1e-7,pow(10,model.predict(X_test))*1e-7))

In [ ]:
# print root mean square error and mean absolute error for testing dataset
mse_test_0 = mse(y_test,model.predict(X_test))
print(mse_test_0**0.5)
print(mae(y_test,model.predict(X_test)))

In [ ]:
mse_train_0 = mse(y_train,model.predict(X_train))
print(mse_train_0**0.5)
print(mae(y_train,model.predict(X_train)))

In [ ]:
print(r2_score(y_train,model.predict(X_train)))
print(r2_score(y_test,model.predict(X_test)))

In [ ]:
mse_train = mse(pow(10,y_train)*1e-7,pow(10,model.predict(X_train))*1e-7)
print(mse_train**0.5)
print(mae(pow(10,y_train)*1e-7,pow(10,model.predict(X_train))*1e-7))

In [ ]:
print(r2_score(pow(10,y_train)*1e-7,pow(10,model.predict(X_train))*1e-7))
print(r2_score(pow(10,y_test)*1e-7,pow(10,model.predict(X_test))*1e-7))

In [ ]:
# plot predicted and ground_truth NO2
plt.scatter(pow(10,y_test)*1e-7,pow(10,model.predict(X_test))*1e-7)
plt.xlabel('NO2_test')
plt.ylabel('NO2_test_predicted')
plt.ylim(0,0.0001)
plt.xlim(0,0.0001)

In [ ]:
#plot predicted and ground_truth log(NO2)
plt.scatter(y_test,model.predict(X_test))
plt.xlabel('log_NO2_test')
plt.ylabel('log_NO2_test_predicted')
plt.ylim(1,3)
plt.xlim(1,3)

In [ ]:
#plot predicted and ground_truth log(NO2) in training set
plt.scatter(y_train,model.predict(X_train))
plt.xlabel('log_NO2_train')
plt.ylabel('log_NO2_train_predicted')
plt.ylim(1,3)
plt.xlim(1,3)

## Error visualisation

In [ ]:
# calculate error in testing dataset
test_error = pd.DataFrame(columns = [])
test_error['log_NO2'] = y_test['log_NO2']
test_error['prediction'] = model.predict(X_test)
test_error['error'] = test_error['prediction'] - test_error['log_NO2']
test_error

In [ ]:
# see error distribution
plt.hist(test_error['error'], bins='auto')
plt.xlabel('log_NO2_test_error')
plt.ylabel('count')
plt.title("Histogram of prediction error")
plt.show()

In [ ]:
test_error.describe()

In [ ]:
# define large error >0.5 or <-0.5
# add a column indicating wether the error is large or not
test_error['large_err'] = test_error.apply(lambda x: (x['error'] < -0.5)|(x['error'] > 0.5), axis = 1)
test_error

In [ ]:
# see error distribution
# large error occus when NO2 is low
fig = px.histogram(test_error, x='log_NO2', color = 'large_err')
fig.show()

## Explain with SHAP

In [ ]:
# use 5000 data (shuffled before during train test split) from training dataset as the explainer
explainer = shap.DeepExplainer(model, X_train[:5000])

In [ ]:
X_train.shape

In [ ]:
# get shap values of the 5000 test data (shuffled before during train test split)
shap_values = explainer.shap_values(X_test[:5000])

In [ ]:
# init the JS visualization code
shap.initjs()
feature_names = before_normalized_X.columns

In [ ]:
# this is global interpretation that represent the averaged impact of each variable
shap.summary_plot(shap_values[0], plot_type = 'bar', feature_names = feature_names)

In [ ]:
# this is global interpretation that show how facebook mobility impact the model output, and how it interacts with apple_driving
shap.dependence_plot('log_facebook_movement', shap_values[0], X_test[:5000], interaction_index = 'apple_driving', feature_names = feature_names)

In [ ]:
shap.dependence_plot('apple_driving', shap_values[0], X_test[:5000], interaction_index = 'apple_walking', feature_names = feature_names)

In [ ]:
model.save('MLP_all_mobility.h5')  
# model = load_model('MLP_all.h5')

### Explain on monthly average for different countries

In [ ]:
merge['date'] =  pd.to_datetime(merge['date'], format='%Y-%m-%d')

In [ ]:
# add in column with year and another with month
merge['month'] = merge['date'].dt.month
merge['year'] = merge['date'].dt.year
merge

In [ ]:
# group by month, agg to mean
merge_by_month_location = merge.groupby(['year','month', 'country','lon','lat']).mean().reset_index()
merge_by_month_location

In [ ]:
merge_by_month_location.to_csv(r'D:\GitHub\NO2-in-South-East-Asia-_GE5219\data\merge_by_month.csv')

In [ ]:
fig = px.scatter(merge_by_month_location[merge_by_month_location['year']==2021], x='month', y='log_facebook_movement', color='log_NO2', size='apple_driving',
                facet_col='country', facet_col_wrap=4)
fig.show()

In [ ]:
fig = px.scatter(merge_by_month_location[merge_by_month_location['country']=='Malaysia'], x='month', y='log_facebook_movement', color='log_NO2', size='apple_driving',
                facet_col='year')
fig.show()

### Explain on Indonesia

In [ ]:
df_ID = merge[merge['country']=='Indonesia']
df_ID

In [ ]:
fig = px.scatter(df_ID, x='date', y='log_NO2', color='log_facebook_movement')
fig.show()

In [ ]:
df_ID_date = df_ID[(df_ID['date']>='2021-03-01')&(df_ID['date']<='2021-03-31')]
df_ID_date

In [ ]:
# X include all input values
before_normalized_X_ID = df_ID_date[['lon','lat','day', 'rainfall','u-wind', 'v-wind', 'wind_speed', 'surface-p', 'dew-pt', '2m-temp', 'facebook_movement', 'log_facebook_movement', 'apple_driving', 'apple_walking', 'haze']]
before_normalized_X_ID

In [ ]:
after_normalized_X_ID = sc_X.transform(before_normalized_X_ID)
after_normalized_X_ID

In [ ]:
shap_values_ID = explainer.shap_values(after_normalized_X_ID)

In [ ]:
shap.summary_plot(shap_values_ID[0], plot_type = 'bar', feature_names = feature_names)

In [ ]:
shap.dependence_plot('log_facebook_movement', shap_values_ID[0], after_normalized_X_ID, interaction_index = 'apple_driving', feature_names = feature_names)

In [ ]:
shap.dependence_plot('apple_walking', shap_values_ID[0], after_normalized_X_ID, interaction_index = 'apple_driving', feature_names = feature_names)

In [ ]:
fig = px.scatter(merge[merge['country']=='Indonesia'], x='date', y='log_facebook_movement', color='log_NO2', hover_data = ['lon','lat'])
fig.show()

### Explain on high NO2

In [ ]:
df2 = merge[(merge['log_NO2']>2.48)&(merge['log_NO2']<2.5)]
df2

In [ ]:
# X include all input values
before_normalized_X_2 = df2[['lon','lat','day', 'rainfall','u-wind', 'v-wind', 'wind_speed', 'surface-p', 'dew-pt', '2m-temp', 'facebook_movement', 'log_facebook_movement', 'apple_driving', 'apple_walking', 'haze']]
df2

In [ ]:
after_normalized_X_2 = sc_X.transform(before_normalized_X_2)
after_normalized_X_2

In [ ]:
shap_values_ID2 = explainer.shap_values(after_normalized_X_2)

In [ ]:
shap.summary_plot(shap_values_ID2[0], plot_type = 'bar', feature_names = feature_names)

In [ ]:
shap.dependence_plot('log_facebook_movement', shap_values_ID2[0], after_normalized_X_2, interaction_index = 'apple_driving', feature_names = feature_names)

In [ ]:
shap.dependence_plot('apple_driving', shap_values_ID2[0], after_normalized_X_2, interaction_index = 'apple_walking', feature_names = feature_names)